<a href="https://colab.research.google.com/github/charan255/charan255/blob/main/ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#to save files models
from google.colab import files

### Save locally first
model.save('model.h5')  # Example for Keras
files.download('model.h5')  # Triggers download

In [1]:
!pip install tensorflow

In [2]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [4]:
trainData = pd.read_csv('customer_churn_dataset-training-master.csv')
trainData.dropna(inplace=True)
trainData

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,2.0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,932.00,17.0,1.0
1,3.0,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,557.00,6.0,1.0
2,4.0,55.0,Female,14.0,4.0,6.0,18.0,Basic,Quarterly,185.00,3.0,1.0
3,5.0,58.0,Male,38.0,21.0,7.0,7.0,Standard,Monthly,396.00,29.0,1.0
4,6.0,23.0,Male,32.0,20.0,5.0,8.0,Basic,Monthly,617.00,20.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
440828,449995.0,42.0,Male,54.0,15.0,1.0,3.0,Premium,Annual,716.38,8.0,0.0
440829,449996.0,25.0,Female,8.0,13.0,1.0,20.0,Premium,Annual,745.38,2.0,0.0
440830,449997.0,26.0,Male,35.0,27.0,1.0,5.0,Standard,Quarterly,977.31,9.0,0.0
440831,449998.0,28.0,Male,55.0,14.0,2.0,0.0,Standard,Quarterly,602.55,2.0,0.0


In [5]:
testData = pd.read_csv('customer_churn_dataset-testing-master.csv')
testData

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,1,22,Female,25,14,4,27,Basic,Monthly,598,9,1
1,2,41,Female,28,28,7,13,Standard,Monthly,584,20,0
2,3,47,Male,27,10,2,29,Premium,Annual,757,21,0
3,4,35,Male,9,12,5,17,Premium,Quarterly,232,18,0
4,5,53,Female,58,24,9,2,Standard,Annual,533,18,0
...,...,...,...,...,...,...,...,...,...,...,...,...
64369,64370,45,Female,33,12,6,21,Basic,Quarterly,947,14,1
64370,64371,37,Male,6,1,5,22,Standard,Annual,923,9,1
64371,64372,25,Male,39,14,8,30,Premium,Monthly,327,20,1
64372,64373,50,Female,18,19,7,22,Standard,Monthly,540,13,1


In [6]:
#preprocessing
trainData = trainData.drop(['CustomerID'], axis=1)
testData = testData.drop(['CustomerID'], axis=1)
testData

,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,22,Female,25,14,4,27,Basic,Monthly,598,9,1
1,41,Female,28,28,7,13,Standard,Monthly,584,20,0
2,47,Male,27,10,2,29,Premium,Annual,757,21,0
3,35,Male,9,12,5,17,Premium,Quarterly,232,18,0
4,53,Female,58,24,9,2,Standard,Annual,533,18,0
...,...,...,...,...,...,...,...,...,...,...,...
64369,45,Female,33,12,6,21,Basic,Quarterly,947,14,1
64370,37,Male,6,1,5,22,Standard,Annual,923,9,1
64371,25,Male,39,14,8,30,Premium,Monthly,327,20,1
64372,50,Female,18,19,7,22,Standard,Monthly,540,13,1


In [7]:
#converting categorical columns into numerical columns
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse_output=False, handle_unknown = 'ignore')
EncodedTrainingData = ohe.fit_transform(trainData[['Gender', 'Subscription Type','Contract Length']])
EncodedTestingData = ohe.transform(testData[['Gender', 'Subscription Type','Contract Length']])

In [8]:
features = ohe.get_feature_names_out(['Gender', 'Subscription Type','Contract Length'])
newTrainingDataRows = pd.DataFrame(EncodedTrainingData, columns=features)
newTestingDataRows = pd.DataFrame(EncodedTestingData, columns=features)
trainData = pd.concat([trainData.drop(['Gender', 'Subscription Type','Contract Length'],axis = 1), newTrainingDataRows], axis = 1)
testData = pd.concat([testData.drop(['Gender', 'Subscription Type','Contract Length'], axis=1),newTestingDataRows], axis=1)

In [9]:
features

array(['Gender_Female', 'Gender_Male', 'Subscription Type_Basic',
       'Subscription Type_Premium', 'Subscription Type_Standard',
       'Contract Length_Annual', 'Contract Length_Monthly',
       'Contract Length_Quarterly'], dtype=object)

In [10]:
trainData

,Age,Tenure,Usage Frequency,Support Calls,Payment Delay,Total Spend,Last Interaction,Churn,Gender_Female,Gender_Male,Subscription Type_Basic,Subscription Type_Premium,Subscription Type_Standard,Contract Length_Annual,Contract Length_Monthly,Contract Length_Quarterly
0,30.0,39.0,14.0,5.0,18.0,932.00,17.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,65.0,49.0,1.0,10.0,8.0,557.00,6.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,55.0,14.0,4.0,6.0,18.0,185.00,3.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,58.0,38.0,21.0,7.0,7.0,396.00,29.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
4,23.0,32.0,20.0,5.0,8.0,617.00,20.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440829,25.0,8.0,13.0,1.0,20.0,745.38,2.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
440830,26.0,35.0,27.0,1.0,5.0,977.31,9.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
440831,28.0,55.0,14.0,2.0,0.0,602.55,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
440832,31.0,48.0,20.0,1.0,14.0,567.77,21.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
testData

,Age,Tenure,Usage Frequency,Support Calls,Payment Delay,Total Spend,Last Interaction,Churn,Gender_Female,Gender_Male,Subscription Type_Basic,Subscription Type_Premium,Subscription Type_Standard,Contract Length_Annual,Contract Length_Monthly,Contract Length_Quarterly
0,22,25,14,4,27,598,9,1,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,41,28,28,7,13,584,20,0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,47,27,10,2,29,757,21,0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
3,35,9,12,5,17,232,18,0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
4,53,58,24,9,2,533,18,0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64369,45,33,12,6,21,947,14,1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
64370,37,6,1,5,22,923,9,1,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
64371,25,39,14,8,30,327,20,1,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
64372,50,18,19,7,22,540,13,1,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [12]:
X_train = trainData.drop('Churn', axis=1)
Y_train = trainData['Churn']
X_test = testData.drop('Churn', axis=1)
Y_test = testData['Churn']

In [13]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(
    X_train, Y_train,
    test_size=0.2,  # 20% for validation
    random_state=42
)

In [14]:
## Scaling
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_val = sc.transform(x_val)
testData = sc.transform(X_test)

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [16]:
model1 = Sequential(
    [
        Dense(128, activation='relu', input_shape=(x_train.shape[1],)),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')
    ]
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
model2 = Sequential(
    [
        Dense(16, activation='relu', input_shape=(x_train.shape[1],)),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ]
)

In [18]:
model3 = Sequential(
    [
        Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ]
)

In [19]:
model4 = Sequential(
    [
        Dense(32, activation='relu', input_shape=(x_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ]
)

In [20]:
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model4.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [21]:
model1.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,689 (73.00 KB)

 Trainable params: 18,689 (73.00 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
model2.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 16)             │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 545 (2.13 KB)

 Trainable params: 545 (2.13 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
model3.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,137 (12.25 KB)

 Trainable params: 3,137 (12.25 KB)

 Non-trainable params: 0 (0.00 B)

In [24]:
model4.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 32)             │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,601 (6.25 KB)

 Trainable params: 1,601 (6.25 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
##Earlt stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [26]:
history1 = model1.fit(x_train, y_train, epochs=50, batch_size=32, validation_data=(x_val, y_val), callbacks=[early_stopping])

Epoch 1/50
11021/11021 ━━━━━━━━━━━━━━━━━━━━ 32s 3ms/step - accuracy: 0.9611 - loss: 0.1096 - val_accuracy: 0.9826 - val_loss: nan
Epoch 2/50
11021/11021 ━━━━━━━━━━━━━━━━━━━━ 41s 3ms/step - accuracy: 0.9822 - loss: 0.0602 - val_accuracy: 0.9826 - val_loss: nan
Epoch 3/50
11021/11021 ━━━━━━━━━━━━━━━━━━━━ 40s 3ms/step - accuracy: 0.9829 - loss: 0.0570 - val_accuracy: 0.9832 - val_loss: nan
Epoch 4/50
11021/11021 ━━━━━━━━━━━━━━━━━━━━ 30s 3ms/step - accuracy: 0.9835 - loss: 0.0549 - val_accuracy: 0.9832 - val_loss: nan
Epoch 5/50
11021/11021 ━━━━━━━━━━━━━━━━━━━━ 29s 3ms/step - accuracy: 0.9840 - loss: 0.0530 - val_accuracy: 0.9855 - val_loss: nan
Epoch 6/50
11021/11021 ━━━━━━━━━━━━━━━━━━━━ 42s 3ms/step - accuracy: 0.9847 - loss: 0.0514 - val_accuracy: 0.9849 - val_loss: nan


In [27]:
history2 = model2.fit(x_train, y_train, epochs=50, batch_size=32, validation_data=(x_val, y_val), callbacks=[early_stopping])

Epoch 1/50
11021/11021 ━━━━━━━━━━━━━━━━━━━━ 30s 3ms/step - accuracy: 0.9240 - loss: 0.1847 - val_accuracy: 0.9785 - val_loss: nan
Epoch 2/50
11021/11021 ━━━━━━━━━━━━━━━━━━━━ 39s 2ms/step - accuracy: 0.9802 - loss: 0.0682 - val_accuracy: 0.9826 - val_loss: nan
Epoch 3/50
11021/11021 ━━━━━━━━━━━━━━━━━━━━ 27s 2ms/step - accuracy: 0.9835 - loss: 0.0597 - val_accuracy: 0.9838 - val_loss: nan
Epoch 4/50
11021/11021 ━━━━━━━━━━━━━━━━━━━━ 42s 3ms/step - accuracy: 0.9842 - loss: 0.0570 - val_accuracy: 0.9846 - val_loss: nan
Epoch 5/50
11021/11021 ━━━━━━━━━━━━━━━━━━━━ 27s 2ms/step - accuracy: 0.9847 - loss: 0.0542 - val_accuracy: 0.9844 - val_loss: nan
Epoch 6/50
11021/11021 ━━━━━━━━━━━━━━━━━━━━ 28s 2ms/step - accuracy: 0.9852 - loss: 0.0517 - val_accuracy: 0.9836 - val_loss: nan


In [28]:
history3 = model3.fit(x_train, y_train, epochs=50, batch_size=32, validation_data=(x_val, y_val), callbacks=[early_stopping])

Epoch 1/50
11021/11021 ━━━━━━━━━━━━━━━━━━━━ 30s 3ms/step - accuracy: 0.9534 - loss: 0.1259 - val_accuracy: 0.9819 - val_loss: nan
Epoch 2/50
11021/11021 ━━━━━━━━━━━━━━━━━━━━ 28s 3ms/step - accuracy: 0.9821 - loss: 0.0611 - val_accuracy: 0.9829 - val_loss: nan
Epoch 3/50
11021/11021 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - accuracy: 0.9836 - loss: 0.0563 - val_accuracy: 0.9834 - val_loss: nan
Epoch 4/50
11021/11021 ━━━━━━━━━━━━━━━━━━━━ 42s 3ms/step - accuracy: 0.9843 - loss: 0.0533 - val_accuracy: 0.9842 - val_loss: nan
Epoch 5/50
11021/11021 ━━━━━━━━━━━━━━━━━━━━ 41s 3ms/step - accuracy: 0.9845 - loss: 0.0521 - val_accuracy: 0.9823 - val_loss: nan
Epoch 6/50
11021/11021 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - accuracy: 0.9849 - loss: 0.0507 - val_accuracy: 0.9844 - val_loss: nan


In [29]:
history4 = model4.fit(x_train, y_train, epochs=50, batch_size=32, validation_data=(x_val, y_val), callbacks=[early_stopping])

Epoch 1/50
11021/11021 ━━━━━━━━━━━━━━━━━━━━ 29s 3ms/step - accuracy: 0.9468 - loss: 0.1415 - val_accuracy: 0.9809 - val_loss: nan
Epoch 2/50
11021/11021 ━━━━━━━━━━━━━━━━━━━━ 27s 2ms/step - accuracy: 0.9826 - loss: 0.0608 - val_accuracy: 0.9836 - val_loss: nan
Epoch 3/50
11021/11021 ━━━━━━━━━━━━━━━━━━━━ 28s 3ms/step - accuracy: 0.9838 - loss: 0.0569 - val_accuracy: 0.9838 - val_loss: nan
Epoch 4/50
11021/11021 ━━━━━━━━━━━━━━━━━━━━ 41s 3ms/step - accuracy: 0.9842 - loss: 0.0543 - val_accuracy: 0.9846 - val_loss: nan
Epoch 5/50
11021/11021 ━━━━━━━━━━━━━━━━━━━━ 28s 3ms/step - accuracy: 0.9846 - loss: 0.0527 - val_accuracy: 0.9838 - val_loss: nan
Epoch 6/50
11021/11021 ━━━━━━━━━━━━━━━━━━━━ 27s 2ms/step - accuracy: 0.9850 - loss: 0.0516 - val_accuracy: 0.9834 - val_loss: nan


In [30]:
##Final Testing
test_predictions1 = model1.predict(testData)
test_classes1 = (test_predictions1 > 0.5).astype("int32")

test_predictions2 = model2.predict(testData)
test_classes2 = (test_predictions2 > 0.5).astype("int32")

test_predictions3 = model3.predict(testData)
test_classes3 = (test_predictions3 > 0.5).astype("int32")

test_predictions4 = model4.predict(testData)
test_classes4 = (test_predictions4 > 0.5).astype("int32")

2012/2012 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
2012/2012 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
2012/2012 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
2012/2012 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step


In [31]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# Basic metrics
print("Accuracy:", accuracy_score(Y_test, test_classes1))
print("Precision:", precision_score(Y_test, test_classes1))
print("Recall:", recall_score(Y_test, test_classes1))
print("F1 Score:", f1_score(Y_test, test_classes1))
print("ROC AUC:", roc_auc_score(Y_test, test_predictions1))

# Confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(Y_test, test_classes1))

Accuracy: 0.5192313667008419
Precision: 0.4962785231612966
Recall: 0.9971140917587643
F1 Score: 0.6627142841574124
ROC AUC: 0.5785467208422749
Confusion Matrix:
[[ 3020 30861]
 [   88 30405]]


In [32]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# Basic metrics
print("Accuracy:", accuracy_score(Y_test, test_classes2))
print("Precision:", precision_score(Y_test, test_classes2))
print("Recall:", recall_score(Y_test, test_classes2))
print("F1 Score:", f1_score(Y_test, test_classes2))
print("ROC AUC:", roc_auc_score(Y_test, test_predictions2))

# Confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(Y_test, test_classes2))

Accuracy: 0.5215925684282474
Precision: 0.4975126006414872
Recall: 0.9970157085232676
F1 Score: 0.6637918800013101
ROC AUC: 0.6139950437549186
Confusion Matrix:
[[ 3175 30706]
 [   91 30402]]


In [33]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# Basic metrics
print("Accuracy:", accuracy_score(Y_test, test_classes3))
print("Precision:", precision_score(Y_test, test_classes3))
print("Recall:", recall_score(Y_test, test_classes3))
print("F1 Score:", f1_score(Y_test, test_classes3))
print("ROC AUC:", roc_auc_score(Y_test, test_predictions3))

# Confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(Y_test, test_classes3))

Accuracy: 0.5194022431416411
Precision: 0.49637001386736274
Recall: 0.9977699799954087
F1 Score: 0.6629406894147383
ROC AUC: 0.5803460394206448
Confusion Matrix:
[[ 3011 30870]
 [   68 30425]]


In [34]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# Basic metrics
print("Accuracy:", accuracy_score(Y_test, test_classes4))
print("Precision:", precision_score(Y_test, test_classes4))
print("Recall:", recall_score(Y_test, test_classes4))
print("F1 Score:", f1_score(Y_test, test_classes4))
print("ROC AUC:", roc_auc_score(Y_test, test_predictions4))

# Confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(Y_test, test_classes4))

Accuracy: 0.519930406685929
Precision: 0.49664363761085795
Recall: 0.997212474994261
F1 Score: 0.6630614914958569
ROC AUC: 0.5902389474060267
Confusion Matrix:
[[ 3062 30819]
 [   85 30408]]
